In [5]:
%matplotlib inline 
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pandas as pd
from PIL import Image, ImageDraw
import sklearn
from os import listdir
from os.path import isfile, join



In [25]:
cat_breed = ["Abyssinian", "Bengal", "Birman", "Bombay", "British_Shorthair", "Egyptian_Mau","Maine_Coon", "Persian", "Ragdoll", "Russian_Blue", "Siamese", "Sphynx"]
cat_breed = [x.lower() for x in cat_breed]

In [27]:
#Load directory of files for image names 
#My dir below is the image directory
mydir = os.getcwd() + "/images/Org_Images"
onlyfiles = [f for f in listdir(mydir) if isfile(join(mydir, f))]

In [28]:
#Create list of files for image name and create list of names
lbl_list = []
image_names = []
for i in onlyfiles:
    #print i.lower().split(".")[-1]
    if  i.lower().split(".")[-1] == "jpg":
        image_names.append(i.split(".")[:-1])
        if i.lower().rpartition("_")[0]  in cat_breed:
            lbl_list.append(1) 
        else:
            lbl_list.append(0) 
    else:
        pass #print i



In [29]:
#Combine label list and image names
a = pd.DataFrame(lbl_list)
b = pd.DataFrame(image_names)
image_list = pd.concat([a, b], axis=1)


In [30]:
#Load boxes from csv that was created from R
box = pd.read_csv("/Users/bobminnich/Documents/Columbia/Courses/Applied_Data_Science/Project3/images/box_list.csv")

In [31]:
box.columns = ["RIndex","title",'xmin', 'ymin', 'xmax', 'ymax']
image_list.columns = ["label","title"]


In [32]:
result = pd.merge(image_list, box, on='title')
#Add X Dimension
result["x_len"] = result["xmax"] - result["xmin"]
#Add Y Dimension
result["y_len"] = result["ymax"] - result["ymin"]
x_median = np.median(result["x_len"])
y_median = np.median(result["y_len"])



In [198]:
#Load Image Directory and create Crop Directory crimdir
imdir = "/Users/bobminnich/Documents/Columbia/Courses/Applied_Data_Science/Project3/images/Org_Images/"
crimdir ="/Users/bobminnich/Documents/Columbia/Courses/Applied_Data_Science/Project3/images/cropped/"
for i in range(0,result.shape[0]):
    image_num = i
    out = result.ix[image_num]
    filename = result.ix[image_num,"title"] + ".jpg"
    outfile = result.ix[image_num,"title"] + ".thumbnail"
    im = Image.open(imdir + filename)
    a = im.crop((out.xmin,out.ymin,out.xmax,out.ymax))
    b = a.resize((155,155))
    b.save(crimdir + filename, "JPEG")

In [18]:
def sifter(image,keypoints):
    sift = cv2.SIFT(keypoints)
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(image,None)
    return des1
def xy_coordinates(kp):
    feature_loc_x = []
    feature_loc_y = []
    for i in kp:
        feature_loc_x.append(i.pt[0])
        feature_loc_y.append(i.pt[1])
    return feature_loc_x,feature_loc_y
def sifter_comare(image1,image2,keypoints):
    sift = cv2.SIFT(keypoints)
    bf = cv2.BFMatcher()
    kp1, des1 = sift.detectAndCompute(image1,None)
    kp2, des2 = sift.detectAndCompute(image2,None)
    matches = bf.knnMatch(des1,des2, k=2)    
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    return kp1, des1,kp2, des2,good

def image_selector(image_number,cropped):
    filename = result.ix[image_number,"title"] + ".jpg"
    if cropped == True:
        imdir ="/Users/bobminnich/Documents/Columbia/Courses/Applied_Data_Science/Project3/images/cropped/"
    else:
        imdir = "/Users/bobminnich/Documents/Columbia/Courses/Applied_Data_Science/Project3/images/Org_Images/"
    img1 = cv2.imread(imdir + filename)
    gray1= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    return gray1

def descript_compare(des1,des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)    
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m.distance)
    return good



In [19]:
im1 = image_selector(2,False)
im2 = image_selector(1000,True)


plt.subplot(121),plt.imshow(im1,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(im2,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])



NameError: global name 'result' is not defined

In [642]:
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [647]:
train_index,test_index = sklearn.cross_validation.train_test_split(result.index,test_size=0.33, random_state=42)

In [668]:
#Create training Descriptors
descriptors = []
for i in train_index:
    image = image_selector(i,True)
    keypoints = 100
    sift = sifter(image,keypoints)
    descriptors.append(sift)

In [687]:
#Create Test Descriptors
descriptors_test = []
for i in test_index:
    image = image_selector(i,False)
    keypoints = 100
    sift = sifter(image,keypoints)
    descriptors_test.append(sift)

In [1047]:
#Create predictions
pred = []
for j in range(0,100): 
    results = []
    results_count = []
    for i in range(0,len(train_index)):
        a =  descript_compare(descriptors[i], descriptors_test[j])
        if len(a) == 0:
            results.append(9999999)
            results_count.append(len(a))
        else:
            results.append(np.median(a))
            results_count.append(len(a))
    a = np.array(results)
    b = np.array(np.argsort(a))
    seq = range(7)
    c = result.ix[b[seq]]
    if (sum(c.label)+0.0)/c.shape[0] >= 0.5 :
        pred.append(1)
    else:
        pred.append(0)

In [1048]:
(sum(pred == result.label[test_index[range(0,100)]])+0.0)/100

0.62